## Inception otherwise known as GoogLeNet Straight out of the paper

In [1]:
%autosave 300
%autoreload 2
%reload_ext autoreload
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir(
    "/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run/code/Users/soutrik.chowdhury/advanced_vision_modules_pytorch"
)
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run/code/Users/soutrik.chowdhury/advanced_vision_modules_pytorch


In [3]:
import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary
from torchview import draw_graph
from pathlib import Path
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random
from torcheval.metrics import MulticlassAccuracy, BinaryAccuracy
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import (
    OneCycleLR,
    StepLR,
    ExponentialLR,
)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import scikitplot as skplt
import seaborn as sns
import torch.nn.functional as F
import albumentations as alb
from albumentations.pytorch import ToTensorV2
import cv2
from torch_lr_finder import LRFinder

In [5]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

In [6]:
def plot_loss_accuracy(
    train_loss,
    val_loss,
    train_acc,
    val_acc,
    labels,
    colors,
    loss_legend_loc="upper center",
    acc_legend_loc="upper left",
    legend_font=5,
    fig_size=(16, 10),
    sub_plot1=(1, 2, 1),
    sub_plot2=(1, 2, 2),
):

    plt.rcParams["figure.figsize"] = fig_size
    plt.figure

    plt.subplot(sub_plot1[0], sub_plot1[1], sub_plot1[2])

    for i in range(len(train_loss)):
        x_train = range(len(train_loss[i]))
        x_val = range(len(val_loss[i]))

        min_train_loss = np.array(train_loss[i]).min()

        min_val_loss = np.array(val_loss[i]).min()

        plt.plot(
            x_train,
            train_loss[i],
            linestyle="-",
            color="tab:{}".format(colors[i]),
            label="TRAIN ({0:.4}): {1}".format(min_train_loss, labels[i]),
        )
        plt.plot(
            x_val,
            val_loss[i],
            linestyle="--",
            color="tab:{}".format(colors[i]),
            label="VALID ({0:.4}): {1}".format(min_val_loss, labels[i]),
        )

    plt.xlabel("epoch no.")
    plt.ylabel("loss")
    plt.legend(loc=loss_legend_loc, prop={"size": legend_font})
    plt.title("Training and Validation Loss")

    plt.subplot(sub_plot2[0], sub_plot2[1], sub_plot2[2])

    for i in range(len(train_acc)):
        x_train = range(len(train_acc[i]))
        x_val = range(len(val_acc[i]))

        max_train_acc = np.array(train_acc[i]).max()

        max_val_acc = np.array(val_acc[i]).max()

        plt.plot(
            x_train,
            train_acc[i],
            linestyle="-",
            color="tab:{}".format(colors[i]),
            label="TRAIN ({0:.4}): {1}".format(max_train_acc, labels[i]),
        )
        plt.plot(
            x_val,
            val_acc[i],
            linestyle="--",
            color="tab:{}".format(colors[i]),
            label="VALID ({0:.4}): {1}".format(max_val_acc, labels[i]),
        )

    plt.xlabel("epoch no.")
    plt.ylabel("accuracy")
    plt.legend(loc=acc_legend_loc, prop={"size": legend_font})
    plt.title("Training and Validation Accuracy")

    plt.show()
    plt.close()

In [7]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""

    def __init__(
        self,
        patience=3,
        verbose=True,
        delta=1e-5,
        trace_func=print,
        path="models",
        model_name="model.pt",
    ):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.model_name = model_name
        self.trace_func = trace_func

    def __call__(self, val_loss, model, epoch):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(
                f"EarlyStopping counter: {self.counter} out of {self.patience}"
            )
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        """Saves model when validation loss decrease."""
        if self.verbose:
            self.trace_func(
                f"Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ..."
            )
        os.makedirs(self.path, exist_ok=True)
        torch.save(model.state_dict(), os.path.join(self.path, self.model_name))
        self.val_loss_min = val_loss


def get_device():
    """Get device (if GPU is available, use GPU)"""
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
# Set manual seed since nn.Parameter are randomly initialzied
set_seed(42)
# Set device cuda for GPU if it's available otherwise run on the CPU
device = get_device()
print(device)
batch_size = 512
epochs = 25
learning_rate = 1e-3

Random seed set as 42
cuda


In [9]:
# Specify the image classes
classes = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

In [21]:
# transform for the dataset using Compose from albumentations
def data_augmentations():
    """Data Augmentations for the CIFAR10 dataset using albumentations library"""
    means = [0.4914, 0.4822, 0.4465]
    stds = [0.2470, 0.2435, 0.2616]

    train_transforms = alb.Compose(
        [
            alb.Normalize(mean=means, std=stds, always_apply=True),
            alb.PadIfNeeded(min_height=40, min_width=40, always_apply=True),
            alb.RandomCrop(height=32, width=32, always_apply=True),
            alb.HorizontalFlip(),
            alb.CoarseDropout(
                max_holes=1,
                max_height=8,
                max_width=8,
                min_holes=1,
                min_height=8,
                min_width=8,
                fill_value=means,
            ),
            ToTensorV2(),
        ]
    )

    test_transforms = alb.Compose(
        [
            alb.Normalize(mean=means, std=stds, always_apply=True),
            ToTensorV2(),
        ]
    )
    return train_transforms, test_transforms

In [22]:
# downloading train and test data
train = datasets.CIFAR10("./data", train=True, download=True)
test = datasets.CIFAR10("./data", train=False, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [23]:
class CifarDS(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        image = self.X[idx]
        label = self.y[idx]
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        return image, label

In [24]:
train_transforms, test_transforms = data_augmentations()
train_dateset = CifarDS(train.data, train.targets, train_transforms)
test_dateset = CifarDS(test.data, test.targets, test_transforms)

In [25]:
def get_data_loader(
    train_dateset, test_dateset, train_transforms, test_transforms, dataloader_args
):
    """Data loader for the CIFAR10 dataset"""
    # Loading custom datasets
    train_dateset = CifarDS(train_dateset.data, train_dateset.targets, train_transforms)
    test_dateset = CifarDS(test_dateset.data, test_dateset.targets, test_transforms)
    # train dataloader
    train_loader = torch.utils.data.DataLoader(train_dateset, **dataloader_args)
    # test dataloader
    test_loader = torch.utils.data.DataLoader(test_dateset, **dataloader_args)
    return train_loader, test_loader

In [26]:
for data, label in train_dateset:
    print(data.shape)
    print(label)
    print(data.mean())
    break

torch.Size([3, 32, 32])
6
tensor(-0.3312)


In [27]:
# Dataloaders
# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = (
    dict(shuffle=True, batch_size=batch_size, num_workers=4, pin_memory=True)
    if device.type == "cuda"
    else dict(shuffle=True, batch_size=batch_size)
)

# train dataloader
train_loader = torch.utils.data.DataLoader(train_dateset, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test_dateset, **dataloader_args)

In [28]:
for images, labels in train_loader:
    print(images.shape)
    print(labels.shape)
    break

/anaconda/envs/torch_env/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/anaconda/envs/torch_env/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


torch.Size([512, 3, 32, 32])
torch.Size([512])


## Inception Lenet Model

https://miro.medium.com/v2/resize:fit:828/format:webp/1*yGcmBL1Nyqpj2lNG2sj5Wg.png

Conv Block for the dataprep part 

In [149]:
class ConvBlock(nn.Module):
    """Convolutional Block for Initial data prep for Inception block"""

    def __init__(self, in_channels, out_channels, **kwargs):
        super(ConvBlock, self).__init__()
        self.convb = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = self.convb(x)
        x = self.bn(x)
        x = F.relu(x)
        return x

Next we design the inception block which is the most important part we throw the model three options: one-by-one, three-by-three, and five-by-five kernels, and we let the model figure out how to weigh and process information from these kernels.

In [139]:
class InceptionBlock(nn.Module):
    """Inception Block for the model"""

    def __init__(self, in_channels, out_11, in_33, out_33, in_55, out_55, out_pool):
        super(InceptionBlock, self).__init__()
        # 1*1 block
        self.branch11 = nn.Conv2d(in_channels, out_11, kernel_size=1)
        # 1*1 -> 3*3 block
        self.branch33 = nn.Sequential(
            nn.Conv2d(in_channels, in_33, kernel_size=1, padding=0),
            nn.Conv2d(in_33, out_33, kernel_size=3, padding=1),
        )
        # 1*1 -> 5*5 block
        self.branch55 = nn.Sequential(
            nn.Conv2d(in_channels, in_55, kernel_size=1, padding=0),
            nn.Conv2d(in_55, out_55, kernel_size=5, padding=2),
        )
        # maxpool -> 1*1 block
        self.branchpool = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, out_pool, kernel_size=1),
        )

    def forward(self, x):
        return torch.cat(
            [self.branch11(x), self.branch33(x), self.branch55(x), self.branchpool(x)],
            1,
        )

In [140]:
inception_block = InceptionBlock(192, 64, 96, 128, 16, 32, 32).to(device)
summary(inception_block, input_size=(1,192, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
InceptionBlock                           [1, 256, 32, 32]          --
├─Conv2d: 1-1                            [1, 64, 32, 32]           12,352
├─Sequential: 1-2                        [1, 128, 32, 32]          --
│    └─Conv2d: 2-1                       [1, 96, 32, 32]           18,528
│    └─Conv2d: 2-2                       [1, 128, 32, 32]          110,720
├─Sequential: 1-3                        [1, 32, 32, 32]           --
│    └─Conv2d: 2-3                       [1, 16, 32, 32]           3,088
│    └─Conv2d: 2-4                       [1, 32, 32, 32]           12,832
├─Sequential: 1-4                        [1, 32, 32, 32]           --
│    └─MaxPool2d: 2-5                    [1, 192, 32, 32]          --
│    └─Conv2d: 2-6                       [1, 32, 32, 32]           6,176
Total params: 163,696
Trainable params: 163,696
Non-trainable params: 0
Total mult-adds (M): 167.62
Input size (MB): 0.79
Forward/ba

Next we will define the auxillary classifiers which will be optional in the main Model

In [141]:
import functools
import operator

In [142]:
class InceptionAux(nn.Module):
    def __init__(self, in_channels, out_classes, input_dim):
        super(InceptionAux, self).__init__()
        self.pool = nn.MaxPool2d(kernel_size=5, stride=3)
        self.convb = nn.Conv2d(in_channels, 128, kernel_size=1, stride=1)
        linear_inchannels = int(128 * (((input_dim - 5) / 3) + 1) ** 2)
        print(linear_inchannels)
        self.fc1 = nn.Linear(linear_inchannels, 1024)
        self.fc2 = nn.Linear(1024, out_classes)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.pool(x)
        print(x.shape)
        x = self.convb(x)
        x = x.reshape(x.shape[0], -1)
        print(x.shape)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=-1)

In [143]:
inception_aux = InceptionAux(128, 10, 32).to(device)
summary(inception_aux, input_size=(1, 128, 32, 32))

12800
torch.Size([1, 128, 10, 10])
torch.Size([1, 12800])


Layer (type:depth-idx)                   Output Shape              Param #
InceptionAux                             [1, 10]                   --
├─MaxPool2d: 1-1                         [1, 128, 10, 10]          --
├─Conv2d: 1-2                            [1, 128, 10, 10]          16,512
├─Linear: 1-3                            [1, 1024]                 13,108,224
├─Dropout: 1-4                           [1, 1024]                 --
├─Linear: 1-5                            [1, 10]                   10,250
Total params: 13,134,986
Trainable params: 13,134,986
Non-trainable params: 0
Total mult-adds (M): 14.77
Input size (MB): 0.52
Forward/backward pass size (MB): 0.11
Params size (MB): 52.54
Estimated Total Size (MB): 53.17

In [144]:
# rand_torch = torch.rand(1, 192, 32, 32).to(device)
# print(rand_torch.flatten(1).shape)
# print(rand_torch.reshape(rand_torch.shape[0], -1).shape)

Next we will define the final model

In [160]:
class InceptionV1(nn.Module):
    """Inception model class"""

    def __init__(self, aux_logits=False, num_classes=10, image_dim=32):
        super(InceptionV1, self).__init__()
        self.aux_logits = aux_logits

        # fierst data prep layer
        self.conv1 = ConvBlock(
            in_channels=3,
            out_channels=64,
            kernel_size=(7, 7),
            stride=(2, 2),
            padding=(3, 3),
        )
        self.conv2 = ConvBlock(64, 192, kernel_size=3, stride=1, padding=1)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # Inception block 1
        self.inception3a = InceptionBlock(192, 64, 96, 128, 16, 32, 32)
        # Inception block 2
        self.inception3b = InceptionBlock(256, 128, 128, 192, 32, 96, 64)
        # Inception block 3
        self.inception4a = InceptionBlock(480, 192, 96, 208, 16, 48, 64)
        # Inception block 4
        self.inception4b = InceptionBlock(512, 160, 112, 224, 24, 64, 64)
        # Inception block 5
        self.inception4c = InceptionBlock(512, 128, 128, 256, 24, 64, 64)
        # Inception block 6
        self.inception4d = InceptionBlock(512, 112, 144, 288, 32, 64, 64)
        # Inception block 7
        self.inception4e = InceptionBlock(528, 256, 160, 320, 32, 128, 128)
        # Inception block 8
        self.inception5a = InceptionBlock(832, 256, 160, 320, 32, 128, 128)
        # Inception block 9
        self.inception5b = InceptionBlock(832, 384, 192, 384, 48, 128, 128)

        # classification block
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(p=0.3)
        self.fc = nn.Linear(1024, num_classes)

        if self.aux_logits:
            # after 3rd inception layer
            self.aux1 = InceptionAux(512, num_classes, image_dim)
            # after 6th inception layer
            self.aux2 = InceptionAux(528, num_classes, image_dim)
        else:
            self.aux1 = self.aux2 = None

    def forward(self, x):
        # data prep layer
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.maxpool(x)
        # inception-1
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool(x)
        x = self.inception4a(x)

        # first aux classifier
        if self.aux_logits and self.training:
            aux1 = self.aux1(x)

        # Inception block 4th
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)

        # second aux classifier
        if self.aux_logits and self.training:
            aux2 = self.aux2(x)

        x = self.inception4e(x)
        x = self.maxpool(x)
        x = self.inception5a(x)
        x = self.inception5b(x)

        # print(x.shape)
        # classification block
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.dropout(x)
        x = self.fc(x)

        if self.aux_logits and self.training:
            return aux1, aux2, x
        return x

In [161]:
inception_model = InceptionV1(aux_logits=False, num_classes=10, image_dim=32).to(device)
summary(inception_model, input_size=(1, 3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
InceptionV1                              [1, 10]                   --
├─ConvBlock: 1-1                         [1, 64, 16, 16]           --
│    └─Conv2d: 2-1                       [1, 64, 16, 16]           9,472
│    └─BatchNorm2d: 2-2                  [1, 64, 16, 16]           128
├─MaxPool2d: 1-2                         [1, 64, 8, 8]             --
├─ConvBlock: 1-3                         [1, 192, 8, 8]            --
│    └─Conv2d: 2-3                       [1, 192, 8, 8]            110,784
│    └─BatchNorm2d: 2-4                  [1, 192, 8, 8]            384
├─MaxPool2d: 1-4                         [1, 192, 4, 4]            --
├─InceptionBlock: 1-5                    [1, 256, 4, 4]            --
│    └─Conv2d: 2-5                       [1, 64, 4, 4]             12,352
│    └─Sequential: 2-6                   [1, 128, 4, 4]            --
│    │    └─Conv2d: 3-1                  [1, 96, 4, 4]             18,5

In [162]:
# updated training module---
def train_module(
    model: torch.nn.Module,
    device: torch.device,
    train_dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    criterion: torch.nn.Module,
    metric,
    train_losses: list,
    train_metrics: list,
):

    # setting model to train mode
    model.train()
    pbar = tqdm(train_dataloader)

    # batch metrics
    train_loss = 0
    train_metric = 0
    processed_batch = 0

    for idx, (data, label) in enumerate(pbar):
        # setting up device
        data = data.to(device)
        label = label.to(device)

        # forward pass output
        preds = model(data)

        # calc loss
        loss = criterion(preds, label)
        train_loss += loss.item()
        # print(f"training loss for batch {idx} is {loss}")

        # backpropagation
        optimizer.zero_grad()  # flush out  existing grads
        loss.backward()  # back prop of weights wrt loss
        optimizer.step()  # optimizer step -> minima

        # metric calc
        preds = torch.argmax(preds, dim=1)
        # print(f"preds:: {preds}")
        metric.update(preds, label)
        train_metric += metric.compute().detach().item()

        # updating batch count
        processed_batch += 1

        pbar.set_description(
            f"Avg Train Loss: {train_loss/processed_batch} Avg Train Metric: {train_metric/processed_batch}"
        )

    # It's typically called after the epoch completes
    metric.reset()
    # updating epoch metrics
    train_losses.append(train_loss / processed_batch)
    train_metrics.append(train_metric / processed_batch)

    return train_losses, train_metrics

In [163]:
# updated testing modules---
def test_module(
    model: torch.nn.Module,
    device: torch.device,
    test_dataloader: torch.utils.data.DataLoader,
    criterion: torch.nn.Module,
    metric,
    test_losses,
    test_metrics,
):
    # setting model to eval mode
    model.eval()
    pbar = tqdm(test_dataloader)

    # batch metrics
    test_loss = 0
    test_metric = 0
    processed_batch = 0

    with torch.inference_mode():
        for idx, (data, label) in enumerate(pbar):
            data, label = data.to(device), label.to(device)
            # predictions
            preds = model(data)
            # print(preds.shape)
            # print(label.shape)

            # loss calc
            loss = criterion(preds, label)
            test_loss += loss.item()

            # metric calc
            preds = torch.argmax(preds, dim=1)
            metric.update(preds, label)
            test_metric += metric.compute().detach().item()

            # updating batch count
            processed_batch += 1

            pbar.set_description(
                f"Avg Test Loss: {test_loss/processed_batch} Avg Test Metric: {test_metric/processed_batch}"
            )

        # It's typically called after the epoch completes
        metric.reset()
        # updating epoch metrics
        test_losses.append(test_loss / processed_batch)
        test_metrics.append(test_metric / processed_batch)

    return test_losses, test_metrics

In [164]:
# driver setup----
# optmizer
optimizer = optim.Adam(inception_model.parameters(), lr=learning_rate)
# loss
criterion = nn.CrossEntropyLoss()
# metric
metric = MulticlassAccuracy(device=device, num_classes=10)
# Early stopping
early_stopping = EarlyStopping(
    patience=5, verbose=True, model_name="inception_base_model.pt"
)

In [165]:
train_losses = []
train_metrics = []
test_losses = []
test_metrics = []

for epoch in range(0, epochs):
    print(f"Epoch {epoch}")
    train_losses, train_metrics = train_module(
        inception_model,
        device,
        train_loader,
        optimizer,
        criterion,
        metric,
        train_losses,
        train_metrics,
    )
    test_losses, test_metrics = test_module(
        inception_model,
        device,
        test_loader,
        criterion,
        metric,
        test_losses,
        test_metrics,
    )

    early_stopping(
        test_losses[-1], inception_model, epoch
    )  # last recorded test loss to measure the improvement against the prior one
    if early_stopping.early_stop:
        print("Early stopping")
        break

Epoch 0


  0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | 0/98 [00:00<?, ?it/s]/

Validation loss decreased (inf --> 1.663784).  Saving model ...
Epoch 1


Avg Train Loss: 1.5194581832204546 Avg Train Metric: 0.37151637430093726: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 98/98 [00:08<00:00, 12.12it/s]


Validation loss decreased (1.663784 --> 1.300805).  Saving model ...
Epoch 2


...

Validation loss decreased (0.596007 --> 0.578573).  Saving model ...
Epoch 23


Avg Train Loss: 0.5770000605558863 Avg Train Metric: 0.8005803561940485: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 98/98 [00:07<00:00, 12.32it/s]


EarlyStopping counter: 1 out of 5
Epoch 24


Avg Train Loss: 0.5604211770150126 Avg Train Metric: 0.8065512964920122: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 98/98 [00:08<00:00, 12.15it/s]


EarlyStopping counter: 2 out of 5
